In [ ]:
# Run this cell to install the required packages
!pip3 install numpy pandas matplotlib seaborn

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

DB_PATH = "./data.db"  # Path to the database file (relative to the current directory)

# Create a connection to the SQLite database
con = sqlite3.connect(DB_PATH)

# Read data from the database into
df_exp = pd.read_sql_query("SELECT * from Experiments", con)
df_meta = pd.read_sql_query("SELECT * from Metadata", con)
df_data = pd.read_sql_query("SELECT * from Singletons", con)

# Close the connection
con.close()

In [ ]:
# Manually peek at the data if needed
df_exp.head()
# df_meta.head()
# df_data.head(20)

Here is the list of the key performance metrics we will collect in our model:
* Application
  1. Offered Load (APP_TX_Rate) - measures how much data is sent by the Sender Application over a certain period of time.
  1. Throughput (APP_RX_Rate) - measures how much data is successfully delivered over a certain period of time between the Sender and Receiver applications. 
  1. Loss Ratio (APP_Loss_Ratio) – measures the ratio of lost datagrams to the total number of datagrams emitted by the sender. A lower value is better as it indicates fewer datagrams were lost and thus a more reliable connection. If this value is zero, then there was no loss at the application level.
  1. Delay (APP_Delay) - measures the time delay for a piece of data to travel from the Sender to the Receiver. When a datagram is sent by the Sender Application, it is tagged with the current time. If the packet is transmitted and received successfully, the Receiver Application calculates the time difference between the current time and the timestamp embedded in the packet. This difference is the end-to-end data delay (also can be referred as latency).
* WiFi MAC
  1. MAC Data TX Rate (MAC_TX_Rate) – measures how much data is sent by the sender’s MAC layer over a certain period of time. 
  1. MAC Data RX Rate (MAC_RX_Rate) – measures how much data is successfully received by the receiver’s MAC layer over a certain period of time.
  1. MAC Loss Ratio (MAC_Loss_Ratio) – measures the ratio of packets lost in transmission to the total number of packets enqueued into the MAC layer on the sender’s side. A lower value is better as it indicates fewer IP packets were lost after being admitted into the MAC layer and thus a more reliable connection. If this value is zero, then there was no loss at the MAC level.
* WiFi PHY
  1. PHY Data TX Rate (PHY_TX_Rate) – measures how much data is sent by the sender’s PHY layer over a certain period of time. Only unicast data MPDUs are taken into consideration.
  1. PHY Data RX Rate (PHY_RX_Rate) – measures how much data is successfully received by the receiver’s PHY layer over a certain period of time. Only unicast data MPDUs are taken into consideration.
  1. PHY Loss Ratio (PHY_Loss_Ratio) – measures the ratio of unicast data MPDUs lost in transmission to the total number of unicast data MPDUs enqueued into the PHY layer on the sender’s side. A lower value is better as it indicates fewer MPDUs were lost in transit and required retransmission. If this value is zero, then there was no loss at the physical level.
  1. PHY Average RSSI (PHY_AVG_RSSI) – measures the average received signal strength indicator (RSSI) of all the received unicast data MPDUs. A higher value is better as it indicates a stronger signal strength. The RSSI is measured in dBm.

Note: Application metrics are collected for each AP-STA pair, while MAC and PHY metrics are collected across all STAs in aggregate.

Payload size presented to MAC layer is APP_SIZE + UDP_HEADER_SIZE + IP_HEADER_SIZE:
macPayloadSize = packetSize + 8 + 20;


In [ ]:
# Calculate metrics

# Step 1: Parse `name` to extract `node_id` and `run`
df_data['node_id'] = df_data['name'].str.extract(r'(node\[\d+\])', expand=False)
df_data['node_id'].fillna('aggregate', inplace=True)  # Assign 'aggregate' to the rows that are not related to any specific node

# Step 2: Pivot the DataFrames
df_data_pivot = df_data.pivot_table(index=['run', 'node_id'], columns='variable', values='value')
df_data_pivot.reset_index(inplace=True)  # Reset the index to make `run` and `node_id` as columns

df_meta_pivot = df_meta.pivot(index='run', columns='key', values='value')
df_meta_pivot = df_meta_pivot.apply(pd.to_numeric, errors='ignore')  # Convert all values to numeric where possible
df_meta_pivot.reset_index(inplace=True)  # Reset the index to make `run` as a column

# Step 3: Merge the metadata (df_meta_pivot) and experiment parameters (df_exp)
df_meta_pivot = pd.merge(df_meta_pivot, df_exp, on='run', how='left')

# Step 4: Sort by elements of column "run"
df_meta_pivot['sort_by'] = df_meta_pivot['run'].str.split('-')
for index, row in df_meta_pivot.iterrows():
    df_meta_pivot.at[index, 'sort_by'] = [int(i) for i in row['sort_by']]
df_meta_pivot.sort_values(by=['sort_by'], inplace=True)
df_meta_pivot.reset_index(drop=True, inplace=True)
df_meta_pivot.drop('sort_by', axis=1, inplace=True)

# Step 5: Merge the two DataFrames
df_data_per_sta = pd.merge(df_meta_pivot, df_data_pivot, on='run', how='left')

# Step 6: Calculate 'delay-average' for aggregate 
# Calculate the average of 'delay-average' for each 'run' where 'node_id' is 'node[x]'
avg_delay_df = df_data_per_sta[df_data_per_sta['node_id'].str.contains('node\[\d+\]')].groupby('run')['delay-average'].mean().reset_index()
# Rename the column in avg_delay_df
avg_delay_df.rename(columns={'delay-average': 'average_node_delay'}, inplace=True)
# Merge this back to the original dataframe df_data_per_sta
df_data_per_sta = pd.merge(df_data_per_sta, avg_delay_df, on='run', how='left')
# Now, for rows where 'node_id' is 'aggregate', replace 'delay-average' with 'average_node_delay'
df_data_per_sta.loc[df_data_per_sta['node_id'] == 'aggregate', 'delay-average'] = df_data_per_sta['average_node_delay']
# We can now drop the 'average_node_delay' column as it has served its purpose
df_data_per_sta.drop('average_node_delay', axis=1, inplace=True)

# if column "distances" exists and not empty (however, they can all be zero, it's valid), then we need to override the distance values in the "distance" column.
# the first value in the distances is the distance for the first node (node[1]), the second value is the distance for the second node (node[2]), and so on.
# if the number of nodes is larger than the number of distances, then the distance fileld should not be replaced for the remaining nodes.
# if the number of nodes is smaller than the number of distances, then the remaining distances should be ignored.
if 'distances' in df_data_per_sta.columns and not df_data_per_sta['distances'].isnull().all():
    # Split the 'distances' column by comma
    df_data_per_sta['distances'] = df_data_per_sta['distances'].str.split(',')
    # Iterate through the rows and replace the values in 'distance_new' with the values from 'distances'
    for index, row in df_data_per_sta.iterrows():
        if row['node_id'] == 'aggregate':
            continue
        else:
            # Get the index of the node
            node_index = int(row['node_id'].split('[')[1].split(']')[0])
            if node_index > len(row['distances']):
                continue
            else:
                df_data_per_sta.at[index, 'distance'] = row['distances'][node_index - 1]

# Step 7: Calculate metrics
df_data_per_sta['app_tx_rate'] = df_data_per_sta['sender-tx-packets'] * df_data_per_sta['packetSize'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['app_rx_rate'] = df_data_per_sta['receiver-rx-packets'] * df_data_per_sta['packetSize'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['app_loss_ratio'] = (df_data_per_sta['sender-tx-packets'] - df_data_per_sta['receiver-rx-packets']) / df_data_per_sta['sender-tx-packets']
df_data_per_sta['app_delay'] = df_data_per_sta['delay-average'] / 1000000  # Convert to ms from ns
df_data_per_sta['mac_payload_size'] = df_data_per_sta['packetSize'] + 28  # 20 for IP header, 8 for UDP header
df_data_per_sta['mac_tx_rate'] = df_data_per_sta['mac-tx-frames'] * df_data_per_sta['mac_payload_size'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['mac_rx_rate'] = df_data_per_sta['mac-rx-frames'] * df_data_per_sta['mac_payload_size'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['mac_loss_ratio'] = (df_data_per_sta['mac-tx-frames'] - df_data_per_sta['mac-rx-frames']) / df_data_per_sta['mac-tx-frames']
df_data_per_sta['phy_tx_rate'] = df_data_per_sta['phy-mpdu-tx-bytes'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['phy_rx_rate'] = df_data_per_sta['phy-mpdu-rx-bytes'] * 8 / df_data_per_sta['duration'] / 1000  # in kbps
df_data_per_sta['phy_loss_ratio'] = df_data_per_sta['phy-mpdu-drop-count'] / df_data_per_sta['phy-mpdu-tx-count']
df_data_per_sta['phy_rssi_avg'] = df_data_per_sta['phy-mpdu-rx-rss-sum'] / df_data_per_sta['phy-mpdu-rx-count']  # in dBm

# Step 8: Calculate average metrics
df_data_per_sta['app_tx_rate_avg'] = df_data_per_sta['app_tx_rate'] / df_data_per_sta['staNum']  # in kbps
df_data_per_sta['app_rx_rate_avg'] = df_data_per_sta['app_rx_rate'] / df_data_per_sta['staNum']  # in kbps
df_data_per_sta['mac_tx_rate_avg'] = df_data_per_sta['mac_tx_rate'] / df_data_per_sta['staNum']  # in kbps
df_data_per_sta['mac_rx_rate_avg'] = df_data_per_sta['mac_rx_rate'] / df_data_per_sta['staNum']  # in kbps
df_data_per_sta['phy_tx_rate_avg'] = df_data_per_sta['phy_tx_rate'] / df_data_per_sta['staNum']  # in kbps
df_data_per_sta['phy_rx_rate_avg'] = df_data_per_sta['phy_rx_rate'] / df_data_per_sta['staNum']  # in kbps

style = {
    'app_tx_rate': '{:.2f} kbps',
    'app_rx_rate': '{:.2f} kbps',
    'app_loss_ratio': '{:.2%}',
    'app_delay': '{:.2f} ms',
    'mac_tx_rate': '{:.2f} kbps',
    'mac_rx_rate': '{:.2f} kbps',
    'mac_loss_ratio': '{:.2%}',
    'phy_tx_rate': '{:.2f} kbps',
    'phy_rx_rate': '{:.2f} kbps',
    'phy_loss_ratio': '{:.2%}',
    'phy_rssi_avg': '{:.2f} dBm',
    'app_tx_rate_avg': '{:.2f} kbps',
    'app_rx_rate_avg': '{:.2f} kbps',
    'mac_tx_rate_avg': '{:.2f} kbps',
    'mac_rx_rate_avg': '{:.2f} kbps',
    'phy_tx_rate_avg': '{:.2f} kbps',
    'phy_rx_rate_avg': '{:.2f} kbps',
    'distance': '{:.2f} m'
}

# Step 9: Save the DataFrame to a CSV file
df_data_per_sta.to_csv('./data.csv', index=False)


In [ ]:
# Print the DataFrame with aggregated metrics ('node_id'=='aggregate')
# Select metrics columns
# app_metrics_columns = ['run', 'input', 'staNum', 'app_tx_rate', 'app_rx_rate', 'app_loss_ratio', 'app_delay']
app_metrics_columns = ['run', 'staNum', "distance", 'app_tx_rate_avg', 'app_rx_rate_avg', 'app_loss_ratio', 'app_delay']

# Select rows where 'node_id' is 'aggregate'
df_app_metrics = df_data_per_sta[df_data_per_sta['node_id'] == 'aggregate'][app_metrics_columns]
# reset the index
df_app_metrics.reset_index(drop=True, inplace=True)

# sort by distance and staNum
df_app_metrics.sort_values(by=['distance', 'staNum'], inplace=True)

# Use df.style to add some nice formatting to the output
styled_df_app_metrics = df_app_metrics.style.format(style)

# Display the DataFrame
display(styled_df_app_metrics)

In [ ]:
# Print the DataFrame with aggregated metrics ('node_id'=='aggregate')
# Select metrics columns
# macphy_metrics_columns = ['run', 'input', 'staNum','mac_tx_rate', 'mac_rx_rate', 'mac_loss_ratio', 'phy_tx_rate', 'phy_rx_rate', 'phy_loss_ratio', 'phy_rssi_avg']
macphy_metrics_columns = ['run', 'staNum', "distance", 'mac_tx_rate_avg', 'mac_rx_rate_avg', 'mac_loss_ratio', 'phy_tx_rate_avg', 'phy_rx_rate_avg', 'phy_loss_ratio', 'phy_rssi_avg']

# Select rows where 'node_id' is 'aggregate'
df_macphy_metrics = df_data_per_sta[df_data_per_sta['node_id'] == 'aggregate'][macphy_metrics_columns]
# reset the index
df_macphy_metrics.reset_index(drop=True, inplace=True)

# sort by distance and staNum
df_macphy_metrics.sort_values(by=['distance', 'staNum'], inplace=True)

# Use df.style to add some nice formatting to the output
styled_df_macphy_metrics = df_macphy_metrics.style.format(style)

# Display the DataFrame
display(styled_df_macphy_metrics)

In [ ]:
# Print the DataFrame with per-user metrics ('node_id'!='aggregate')
# Select metrics columns
app_metrics_columns = ['run', 'input', 'node_id', 'app_tx_rate', 'app_rx_rate', 'app_loss_ratio', 'app_delay']

# Select rows where 'node_id' is 'aggregate'
df_app_metrics = df_data_per_sta[df_data_per_sta['node_id'] != 'aggregate'][app_metrics_columns]
# reset the index
df_app_metrics.reset_index(drop=True, inplace=True)

# Use df.style to add some nice formatting to the output
styled_df_app_metrics = df_app_metrics.style.format(style)

# Display the DataFrame
# display(styled_df_app_metrics)


# Display separate output per run_id of run column
# for run_id in df_app_metrics['run'].unique():
#     display(df_app_metrics[df_app_metrics['run'] == run_id].style.format(style))


In [ ]:
def plot_data(df, x_var, y_var, plot_type='scatter', x_label=None, y_label=None, title=None, legend=True, hue=None, palette='bright'):
    """
    Plot data from a DataFrame.
    
    Args:
        df: DataFrame.
        x_var: Name of the column to use as the x variable in the plot.
        y_var: Name of the column to use as the y variable in the plot.
        plot_type: Type of plot to generate. Default is 'scatter'.
    """
    # Specify the figure size and resolution
    plt.figure(figsize=(5, 3), dpi=150)

    # Generate the plot
    if plot_type == 'scatter':
        ax = sns.scatterplot(data=df, x=x_var, y=y_var, hue=hue, legend=legend, palette=palette)
    elif plot_type == 'line':
        ax = sns.lineplot(data=df, x=x_var, y=y_var, hue=hue, legend=legend, palette=palette)
         # Add dots at each data point
        sns.scatterplot(data=df, x=x_var, y=y_var, hue=hue, legend=False, palette=palette)
    elif plot_type == 'box':
        ax = sns.boxplot(data=df, x=x_var, y=y_var, hue=hue, palette=palette)
    elif plot_type == 'kde':
        ax = sns.kdeplot(data=df, x=x_var, y=y_var, hue=hue, fill=True, palette=palette)
    elif plot_type == 'bar':
        ax = sns.barplot(data=df, x=x_var, y=y_var, hue=hue, palette=palette)
        legend = False  # bar plots typically don't need a legend
        if hue == x_var:
            # Get a color map
            cmap = sns.color_palette("husl", len(df[x_var].unique()))

            # Create a dictionary mapping node_id to color
            color_dict = dict(zip(df[x_var].unique(), cmap))

            # Make the barplot without hue, but with colors mapped according to the dictionary
            ax = sns.barplot(data=df, x=x_var, y=y_var, palette=color_dict)
        else:
            ax = sns.barplot(data=df, x=x_var, y=y_var, hue=hue, palette=palette)
    else:
        raise ValueError(f'Unknown plot_type: {plot_type}')

    # Add grid
    ax.grid(True, zorder=0)

    # Set zorder for the plots so they are above the grid
    if plot_type in ['scatter', 'line']:
        for c in ax.collections:
            c.zorder = 10

    # Add labels and title
    x_label = x_var if x_label is None else x_label
    y_label = y_var if y_label is None else y_label
    title = f'{y_var} vs {x_var}' if title is None else title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    
    # Remove legend if disabled (using hue creates a legend by default)
    if not legend:
        plt.legend().remove()
    else:
        # Move legend to the side of the plot
        ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
        
    # Show the plot
    plt.tight_layout()
    plt.show()


In [ ]:
# plot_data(df_data_per_sta, 'staNum', 'app_rx_rate_avg', 'line', x_label="Number of STAs", y_label="app_rx_rate_avg (kbps)", legend=False)
plot_data(df_data_per_sta[df_data_per_sta['node_id'] == 'aggregate'], 'distance', 'app_rx_rate_avg', 'line', x_label="distance (m)", y_label="app_rx_rate_avg (kbps)", hue="staNum")


In [ ]:
# plot_data(df_data_per_sta, 'node_id', 'app_loss_ratio', 'bar', x_label="node_id", y_label="app_loss_ratio", legend=False)
plot_data(df_data_per_sta[df_data_per_sta['node_id'] == 'aggregate'], 'staNum', 'app_loss_ratio', 'line', x_label="Number of STAs", y_label="app_loss_ratio", legend=False)


In [ ]:
# plot_data(df_data_per_sta, 'node_id', 'app_delay', 'bar', x_label="node_id", y_label="app_delay (ms)", legend=False)
plot_data(df_data_per_sta[df_data_per_sta['node_id'] == 'aggregate'], 'staNum', 'app_delay', 'line', x_label="Number of STAs", y_label="app_delay (ms)", legend=False)


Some Path Loss and RSSI calculations

In [ ]:
# Constants
c = 3e8  # Speed of light
d = np.arange(1, 101, 1)  # Distance array from 1 to 100 meters

# Frequencies
freq_24_GHz = 2.4e9
freq_515_GHz = 5.15e9

# Friis path loss formula calculation
PL_24_GHz = 20 * np.log10(freq_24_GHz) + 20 * np.log10(4 * np.pi / c) + 20 * np.log10(d)
PL_515_GHz = 20 * np.log10(freq_515_GHz) + 20 * np.log10(4 * np.pi / c) + 20 * np.log10(d)

# Plotting
plt.figure(figsize=(5, 3), dpi=150)
plt.plot(d, PL_24_GHz, label='2.4 GHz')
plt.plot(d, PL_515_GHz, label='5.15 GHz')
plt.xlabel('Distance (m)')
plt.ylabel('Path Loss (dB)')
plt.legend()
plt.grid(True)
plt.title('Path Loss vs Distance at 2.4 GHz and 5.15 GHz')
plt.show()

In [ ]:

# Constants
tx_power_dbm_values = [16, 20] # Transmit power in dBm values
tx_gain_dbi = 0 # Transmit antenna gain in dBi
rx_gain_dbi = 0 # Receive antenna gain in dBi
pl_exp = 3.0 # Path loss exponent

# Frequencies for 2.4 GHz and 5.15 GHz Wi-Fi
frequencies = [2.4e9, 5.15e9]
freq_labels = ['2.4 GHz', '5.15 GHz']

# Generate distances from 1 to 100 meters
distances = np.arange(1, 101, 1)

# Create 2 plots with larger figure size and DPI
fig, axs = plt.subplots(2, figsize=(5, 5), dpi=150)

# Calculate RSS for each distance, frequency and power level
for freq, ax, label in zip(frequencies, axs, freq_labels):
    for tx_power_dbm in tx_power_dbm_values:
        ref_loss = 20 * np.log10(freq) + 20 * np.log10(4 * np.pi / 3e8) # Friis free-space model
        path_loss = ref_loss + 10 * pl_exp * np.log10(distances)
        rss = tx_power_dbm + tx_gain_dbi + rx_gain_dbi - path_loss
        ax.plot(distances, rss, label=f'TX Power {tx_power_dbm} dBm')

    ax.axhline(-80, color='r', linestyle='--')  # Add horizontal line at -80 dBm
    ax.set_title(label)
    ax.set_xlabel('Distance (m)')
    ax.set_ylabel('RSS (dBm)')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()